In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Psoriatic_Arthritis/GSE69366'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Autoantibodies and nucleic acids skew complement consumption in systemic lupus erythematosus [C4]"
!Series_summary	"Systemic lupus erythematosus is a chronic autoimmune disease with multifactorial ethiopathogenesis. The complement system is involved in both the early and late stages of disease development and organ damage. To better understand autoantibody mediated complement consumption the GAPAID consortium examined ex vivo immune complex formation on autoantigen arrays.  We recruited patients with SLE (n=211), with other systemic autoimmune diseases (n=65) and non-autoimmune control subjects (n=149) in two rheumatology tertiary care centers. Standard clinical and laboratory data were collected from all subjects and serum complement levels were determined in SLE patients. The genotype of SNP rs1143679 in the ITGAM gene was also determined. On-chip formation of immune complexes was examined using a functional immunoassay on autoantigen microarray

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if gene expression data is available
is_gene_available = True  # Assuming from context

# Check for the availability and conversion of Psoriatic Arthritis information
trait_row = 1 if any("psa-psoriatic arthritis" in entry.lower() for entry in sample_characteristics_dict[1]) else None

def convert_trait(value):
    try:
        val = value.split(":")[1].strip().lower()
        return 1 if "psa-psoriatic arthritis" in val else 0
    except:
        return None

# Check for the availability and conversion of age information
age_row = 2 if len(set(sample_characteristics_dict[2])) > 1 else None

def convert_age(value):
    try:
        return float(value.split(":")[1].strip())
    except:
        return None

# Check for the availability and conversion of gender information
gender_row = 0 if len(set(sample_characteristics_dict[0])) > 1 else None

def convert_gender(value):
    try:
        val = value.split(":")[1].strip().lower()
        return 1 if val == "male" else 0 if val == "female" else None
    except:
        return None

# Save cohort information
save_cohort_info('GSE69366', './preprocessed/Psoriatic_Arthritis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Psoriatic_Arthritis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Psoriatic_Arthritis/trait_data/GSE69366.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1699040': [0.0, 34.0, 0.0], 'GSM1699041': [0.0, 31.0, 0.0], 'GSM1699042': [0.0, 38.0, 0.0], 'GSM1699043': [0.0, 30.0, 0.0], 'GSM1699044': [0.0, 23.0, 1.0], 'GSM1699045': [0.0, 43.0, 0.0], 'GSM1699046': [0.0, 31.0, 0.0], 'GSM1699047': [0.0, 48.0, 0.0], 'GSM1699048': [0.0, 28.0, 0.0], 'GSM1699049': [0.0, 25.0, 1.0], 'GSM1699050': [0.0, 30.0, 0.0], 'GSM1699051': [0.0, 68.0, 0.0], 'GSM1699052': [0.0, 39.0, 0.0], 'GSM1699053': [0.0, 49.0, 0.0], 'GSM1699054': [0.0, 28.0, 0.0], 'GSM1699055': [0.0, 55.0, 0.0], 'GSM1699056': [0.0, 61.0, 0.0], 'GSM1699057': [0.0, 47.0, 1.0], 'GSM1699058': [0.0, 70.0, 1.0], 'GSM1699059': [0.0, 19.0, 0.0], 'GSM1699060': [0.0, 53.0, 1.0], 'GSM1699061': [0.0, 75.0, 0.0], 'GSM1699062': [0.0, 35.0, 0.0], 'GSM1699063': [0.0, 67.0, 0.0], 'GSM1699064': [0.0, 42.0, 1.0], 'GSM1699065': [0.0, 27.0, 0.0], 'GSM1699066': [0.0, 57.0, 0.0], 'GSM1699067': [0.0, 73.0, 0.0], 'GSM1699068': [0.0, 80.0, 1.0], 'GSM1699069': [0.0, 71.0, 1.0], 'GSM1699070': [0.0, 51.0, 0.0], 'GSM169

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'SPOT_ID': ['BB', 'BB', 'PBS', 'hIgG{4}', 'hIgG{4}'], 'Full_Name': ['Fluorescent positioning spot', 'Fluorescent positioning spot', 'PBS', 'Human IgG Purified Immunoglobulin', 'Human IgG Purified Immunoglobulin'], 'Printed conc. (mg/ml)': [nan, nan, nan, '0.018519mg/ml', '0.018519mg/ml'], 'Supplier': [nan, nan, nan, 'Sigma', 'Sigma'], 'Cat.No.': [nan, nan, nan, 'I2511', 'I2511'], 'Lot.No.': [nan, nan, nan, '100M4830', '100M4830'], 'UniProt entry': [nan, nan, nan, nan, nan]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine the appropriate keys for identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Full_Name'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library.
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Psoriatic_Arthritis/gene_data/GSE69366.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Psoriatic_Arthritis')

# 4. Save the cohort information.
save_cohort_info('GSE69366', './preprocessed/Psoriatic_Arthritis/cohort_info.json', True, True, trait_biased, merged_data)

# 5. If the trait is not severely biased, save the merged data to a csv file.
if not trait_biased:
    csv_path = './preprocessed/Psoriatic_Arthritis/GSE69366.csv'
    unbiased_merged_data.to_csv(csv_path)


No gene data in the dataframe
